<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/122_%E5%9E%8B%E4%BB%98%E3%81%8D%E3%83%87%E3%83%BC%E3%82%BF%E6%A7%8B%E9%80%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

型付きデータ構造
================

NamedTuple
----------

NamedTuple（名前付きタプル）は `tuple` のサブクラスであり、各要素にドット `.` 演算子と名前を使った方法でも参照することができるように拡張された型である。 NamedTuple の要素をフィールドと呼ぶ。

NamedTuple は `tuple` のサブクラスなので、以下の `tuple` の特徴を全て持つ。

  * `[]` を使った添字表記によるインデックス参照
  * アンパックした代入
  * イミュータブル
  * ハッシュ可能
  * イテラブル

NamedTuple のインスタンスは、インスタンスごとの属性辞書を持たないので軽量で、普通のタプル以上のメモリを使用しない。

ただし、 NamedTuple という名前のサブクラスが定義されているわけではない。 NamedTuple となる型は、組み込みクラス `type` のラッパーである `typing.NamedTuple` を継承する形で定義する。各フィールドは、型アノテーション付きのクラス変数の宣言という形で定義する。フィールドの順番は、クラス変数を定義した順番と一致する。

In [ ]:
from typing import NamedTuple

class Employee(NamedTuple):
    name: str
    id: int

if __name__ == "__main__":
    employee = Employee("Guido", 3)
    print(f"{employee=}")
    assert isinstance(employee, tuple)  # tuple のサブクラスのインスタンスである
    assert employee.id == employee[1] == 3  # 名前でもインデックスでもフィールドを参照できる
    name, _ = employee  # アンパック
    assert name == "Guido"
    try:
        employee.id = 30
    except AttributeError:
        print("名前付きタプルはイミュータブルである")
    import sys
    print(f"{sys.getsizeof(employee)=}")  # 名前付きタプルのメモリ割り当ての大きさを調べる
    print(f"{sys.getsizeof(('Guido', 3))=}")  # 普通のタプルのメモリ割り当ての大きさを調べる

employee=Employee(name='Guido', id=3)
名前付きタプルはイミュータブルである
sys.getsizeof(employee)=56
sys.getsizeof(('Guido', 3))=56


フィールドにデフォルト値を与えるには、クラス変数の値を定義する。ただし、デフォルト値のあるフィールドはデフォルト値のないフィールドの後でなければならない。これは、関数の定義にデフォルト値付き引数を設定する場合と同様である。

In [ ]:
from typing import NamedTuple

class Employee(NamedTuple):
    name: str
    id: int
    title: str = ""

if __name__ == "__main__":
    employee = Employee("Guido", 3)
    assert employee.title == ""

NamedTuple のサブクラスは docstring やメソッドも持てる。

Python 3.12 からは、ジェネリック型と NamedTuple を組み合わせることもできる。

``` python
class Group[T](NamedTuple):
    """Represents a group."""
    key: T
    group: list[T]
```

実は、 `typing.NamedTuple` の定義内では、クラスを動的に定義するために、 `type()` を直接呼び出すのではなく、 `collections.namedtuple()` を呼び出しており、この関数が `type()` を呼び出している。

`collections.namedtuple()` をファクトリ関数として利用することもできある。

``` python
collections.namedtuple(typename, field_names, *, rename=False, defaults=None, module=None)
```

| 引数 | 意味 |
|:--:|:---|
| `typename` | 作成する NamedTuple の型名を指定する |
| `field_names` | フィールドに付ける名前を指定する。`['x', 'y']` のような文字列のシーケンス、あるいは、`'x y'` や `'x, y'` のように空白やカンマ `,`、`/` で区<br />切った文字列とする。数字やアンダースコア `_` で始まる名前、Python の予約語を使うことはできない |
| `rename` | 真の場合、数字やアンダースコア `_` で始まる名前や Python の予約語を使ったフィールド名を、自動的に `_0` のような位置を示す名前に置き換える |
| `defaults` | イテラブルを指定すると、後ろのフィールドのデフォルト値を設定できる。たとえば、`field_names` が `['x', 'y', 'z']` で `defaults` が `(1, 2)` の<br />場合、 `x` は必須の引数、 `y` は `1` がデフォルト、 `z` は `2` がデフォルトとなる |
| `module` | NamedTuple の `__module__` 属性を指定された値に設定する |

`collections.namedtuple()` をファクトリ関数に使用して NamedTuple 型を作成する場合、 `typing.NamedTuple` にはないオプション指定が可能であるが、各フィールドに型アノテーションを付けることができないので静的型チェックの恩恵を受けられない。

上記の `Employee` 型（デフォルト値付き）は、 `collections.namedtuple()` を使用して以下のように作成することもできる:

In [ ]:
import collections
Employee = collections.namedtuple('Employee', 'name, id, title', defaults=('',))
employee = Employee("Guido", 3)
employee

Employee(name='Guido', id=3, title='')

NamedTuple は `tuple` から継承したメソッドに加えて、以下に示す 2 つの属性と 3 つの追加メソッドをサポートしている。フィールド名との衝突を避けるために、属性名とメソッド名はアンダースコアで始まる。

| 属性 | 意味 |
|:---|:---|
| `_fields` | フィールド名をリストにしたタプル |
| `_field_defaults` | フィールド名からデフォルト値への対応を持つ辞書 |

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `_make(iterable)` | 既存のシーケンスやイテラブルから新しいインスタンスを作るクラスメソッド | NamedTuple |
| `_asdict()` | フィールド名を対応する値にマッピングする新しい辞書を返す | `dict` |
| `_replace(**kwargs)` | 指定されたフィールドを新しい値で置き換えた、新しいインスタンスを作って返す。<br />この機能は Python 3.13 以降で `copy.replace()` 関数にもサポートされる | NamedTuple |

TypedDict
---------

`typing.TypedDict` を利用すると、`dict` 型について、キーごとに値の型を指定することができる。`dict` で型付けするよりも、より厳密に型付けできる。`typing.TypedDict` の使い方は、継承を利用して、クラス変数とその型アノテーションによってキーと値の型付けを指定する:

In [ ]:
from typing import TypedDict

class Point2D(TypedDict):
    x: int
    y: int
    label: str

if __name__ == "__main__":
    a: Point2D = {"x": 1, "y": 2, "label": "good"}          # OK
    b: Point2D = {"x": 1, "y": 2}                           # キーが欠損していると、静的型チェッカーはエラーを出力する
    c: Point2D = {"x": 1, "y": 2, "z": 3, "label": "good"}  # 未定義のキーがあると、静的型チェッカーはエラーを出力する
    d: Point2D = {"x": 1, "y": 2, "label": ["bad"]}         # 値の型が違うと、静的型チェッカーはエラーを出力する

`typing.TypedDict` では、クラス変数の名前がキーの名前になる。このため、キーに Python の予約語を使ったり、ハイフン `-` を含めたりすることができない（この制限は `dict()` コンストラクタをキーワード引数を指定する形で呼び出す場合と同様である）。

`typing.TypedDict` の使い方にはもう 1 つの方法があり、次のようにファクトリ関数として型を定義する:

In [ ]:
from typing import TypedDict

Point2D = TypedDict('Point2D', {'x': int, 'y': int, 'label': str})

if __name__ == "__main__":
    a: Point2D = {"x": 1, "y": 2, "label": "good"}          # OK

この方法では、キーに Python の予約語を使うことも、ハイフン `-` を含めたりすることも可能である。

`typing.TypedDict` を使って定義されたクラスオブジェクトは呼び出し可能で、`__call__ = dict` と定義されている。このため、そのクラスを `dict` の別名のように使うことができる。

In [ ]:
assert Point2D(x=1, y=2, label="first") == dict(x=1, y=2, label="first")

`typing.TypedDict` を利用して指定したキーが辞書に存在しないとき、静的型チェッカーはエラーを出力する。`total=False` 引数をつけて`TypedDict` を継承すると、指定したキーが辞書に存在しないときでも、静的型チェッカーはエラーを出力しない。

In [ ]:
from typing import TypedDict

class Point2D(TypedDict, total=False):
    x: int
    y: int
    label: str

if __name__ == "__main__":
    a: Point2D = {'x': 1, 'y': 2}  # OK
    b: Point2D = {}                # OK

Python 3.11 からは、省略可能なオプションキーと、省略できないキーを明示的に指定することができる。

In [ ]:
from typing import TypedDict, NotRequired, Required

class Point2D(TypedDict):
    x: int
    y: int
    label: NotRequired[str]  # 省略可

class Address(TypedDict, total=False):
    addr1: Required[str]  # 省略不可
    addr2: str  # 省略可
    addr3: str  # 省略可

if __name__ == "__main__":
    po_a: Point2D = {"x": 1, "y": 2}      # OK
    addr_b: Address = {"addr1": "東京都"}  # OK
    addr_c: Address = {"addr2": "千代田区", "addr3": "千代田"}  # キーが欠損していると、静的型チェッカーはエラーを出力する

Python 3.12 からは、ジェネリック型と `typing.TypedDict` を組み合わせることもできる。

``` python
class Group[T](TypedDict):
    key: T
    group: list[T]
```

Python 3.13 からは、読み取り専用のアイテムを指定できるようになった。

``` python
from typing import TypedDict, ReadOnly

class FooDict(TypedDict):
    x: int
    y: ReadOnly[int]  # y は読み取り専用

if __name__ == "__main__":
    foo: FooDict = {"x": 1, "y": 2}
    foo["x"] = 4  # Ok
    foo["y"] = 5  # 読み取り専用アイテムの値を変更すると、静的型チェッカーはエラーを出力する
```

dataclass
---------

### 概要 ###

標準ライブラリの `dataclasses` モジュールは、クラスデコレーター `dataclasses.dataclass` を提供する。これを利用すると、C 言語の構造体のような、複数のデータ型をまとめて取り扱うデータ構造が簡単に定義できる。このデータ構造を **dataclass（データクラス）**と呼び、格納するデータを**フィールド**（field）と呼ぶ。TypedDict でも同様のデータ構造を定義できるが、dataclass のほうが機能が豊富なので、どうしても辞書を利用したいという場合でない限り、TypedDict より dataclass を使うとよい。

dataclass のフィールドは、型アノテーション付きのクラス変数宣言の形式で定義される。

``` python
from dataclasses import dataclass

@dataclass
class User:
    name: str
    active: bool
```

このクラス定義は、`dataclass()` デコレーターにより `__init__()` メソッドと `__repr__()` メソッド、`__eq__()` メソッド、および `__match_args__` 属性が自動的に生成される結果、次のクラス定義とほぼ同等になる（ただし `__match_args__` 属性の生成は Python 3.10 から）。

``` python
class User:
    __match_args__ = ('name', 'active')

    def __init__(self, name: str, active: bool):
        self.name = name
        self.active = active

    def __repr__(self):
        return f"User(name='{self.name}', active={self.active})"

    def __eq__(self, other: object):
        if other.__class__ is self.__class__:
            return (self.name, self.active) == (other.name, other.active)
        return NotImplemented
```

`tabulate` は、 dataclass インスタンスのリストにも対応しており、表形式に整形して出力する。

In [4]:
from dataclasses import dataclass
from tabulate import tabulate

@dataclass
class User:
    name: str
    active: bool

print(tabulate([User("Alice", True), User("Bob", False)],
               headers=["Name", "Active"]))

Name    Active
------  --------
Alice   True
Bob     False


### フィールドのデフォルト値 ###

`__init__()` メソッドの引数として現れるフィールドの順序は、それらのフィールドがクラス定義に現れた順序になる。デフォルト値を設定したフィールドは、`__init__()` メソッドのデフォルト値付き引数となる。

``` python
@dataclass
class User:
    name: str
    active: bool = False
```

このフィールド宣言に対して、`dataclass()` は次のような `__init__()` メソッドを生成する。

``` python
def __init__(self, name: str, active: bool = False):
    self.name = name
    self.active = active
```

デフォルト値を設定する場合は、デフォルト値を持たないフィールドよりも下に宣言する必要がある。これは、関数の定義にデフォルト値付き引数を設定する場合と同様である。

デフォルト値付きフィールドは、クラス変数宣言ではなくクラス変数定義となるので、一般のクラスでのクラス変数と同様にアクセス可能である。デフォルト値付きフィールドにアクセスして値を変更しても、その dataclass の動作に影響を与えることはない。 dataclass 定義の実行と同時に `__init__()` メソッドなどが生成されるからである。デフォルト値付きフィールドへの上書きは、混乱するだけなのでしないこと。

`list` や `set`、`dict` などミュータブルな型のフィールドにデフォルト値を設定する場合は、`dataclasses.field()` 関数を使用する必要がある。

``` python
dataclasses.field(*, default=MISSING, default_factory=MISSING, init=True, repr=True, hash=None, compare=True, metadata=None, kw_only=MISSING)
```

この関数は、フィールドごとに初期化の指示を与える。フィールドのデフォルト値を置く場所に、それを置き換える形でこの関数が呼び出されるため、デフォルト値を設定する場合は、`default` 引数または `default_factory` 引数の 1 つだけを渡す必要がある（両方渡すとエラーが発生する）。これらはデフォルトでモジュール定数 `dataclasses.MISSING` となっている。両方とも `dataclasses.MISSING` のままなら、`dataclasses.field()` 関数は、フィールドがクラス変数としてアクセスされないようにクラス属性からフィールドを削除する。コードで `dataclasses.MISSING` 値を直接使用してはならない。

この関数の引数は、すべてキーワード専用である。

| 引数 | 意味 |
|:---|:---|
| `default` | このフィールドのデフォルト値 |
| `default_factory` | このフィールドにデフォルト値が必要なときに呼び出される、引数なしの呼び出し可能オブジェクト |
| `init` | `True`（デフォルト）の場合、このフィールドは、生成された `__init__()` メソッドの引数として含まれる |
| `repr` | `True`（デフォルト）の場合、このフィールドは、生成された `__repr__()` メソッドによって返される文字列に含まれる |
| `hash` | `True` の場合、このフィールドは生成された `__hash__()` メソッドに含まれる。`None`（デフォルト）の場合、`compare` 引数の値を使用する。<br />通常 `hash` の値は `None` でよい |
| `compare` | `True`（デフォルト）の場合、このフィールドは生成された等価性および比較メソッド（`__eq__()`、`__gt__()` など）に含まれる |
| `metadata` | この引数はサードパーティの拡張メカニズムとして提供される |
| `kw_only` | `True` の場合、このフィールドはキーワード専用としてマークされる。これは、生成された `__init__()` メソッドの引数が計算されるとき<br />に使用される。Python 3.10 で追加 |

たとえば、フィールドにデフォルト値として空のリストを設定する場合は、次のようになる:

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Mydataclass:
    mylist: list[int] = field(default_factory=list)

Mydataclass()

Mydataclass(mylist=[])

`default_factory` 引数には、引数なしで呼び出し可能なオブジェクトを指定しなければならない。たとえば、次は無名関数を使って空でない特定の要素を持つリストとしてデフォルト値を設定する:

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Mydataclass:
    mylist: list[int] = field(default_factory=lambda: [1, 2, 3])

Mydataclass()

Mydataclass(mylist=[1, 2, 3])

このフィールドのデフォルト値は、コンストラクタを呼び出すたびに `default_factory` により生成されたオブジェクトの値となるので、その変更はインスタンスの間で共有されない。

In [ ]:
m1, m2 = Mydataclass(), Mydataclass()
m1.mylist.append(4)
assert m1.mylist == [1, 2, 3, 4]
assert m2.mylist == [1, 2, 3]

フィールドを `repr()` の出力から削除するために `dataclasses.field()` 関数を使用することもできる。たとえば

In [ ]:
from dataclasses import dataclass, field

@dataclass
class C:
    x: int
    y: int = field(repr=False)
    z: int = field(repr=False, default=10)
    t: int = 20

c = C(1, 2)
assert (c.x, c.y, c.z, c.t) == (1, 2, 10, 20)
print(c)

C(x=1, t=20)


### キーワード専用フィールド ###

Python 3.10 からは、 `dataclasses.field()` 関数でフィールドごとにキーワード専用とマークすることができる。キーワード専用とされたフィールドは、`__init__()` メソッドでキーワード専用引数として現れる。

In [ ]:
from dataclasses import dataclass, field

@dataclass
class C:
    x: int
    flag: bool = field(kw_only=True, default=True)

try:
    c = C(1, False)
except Exception as e:
    print(f"{type(e).__name__}: {e}")

TypeError: C.__init__() takes 2 positional arguments but 3 were given


### 型アノテーションを付与しない場合 ###

型アノテーションを付けないで定義した属性は、`__init__()` メソッドの引数に追加されず、ただのクラス変数扱いとなる。

In [ ]:
from dataclasses import dataclass

@dataclass
class User:
    name: str
    active: bool = False
    group = 1  # クラス変数となる

User.group = 2
print(f"{User('hoge').group=}")

try:
    print(User(name="foo", active=True, group=2))  # インスタンス生成時に group を指定するとエラー
except Exception as e:
    print(f"{type(e).__name__}: {e}")

User('hoge').group=2
TypeError: User.__init__() got an unexpected keyword argument 'group'


### dataclass の比較 ###

自動的に生成される `__eq__()` メソッドによって、dataclass オブジェクト同士の比較 `==` は、各フィールドの値が一致するなら `True` を返し、そうでないなら `False` を返す。

In [ ]:
from dataclasses import dataclass

@dataclass
class User:
    name: str
    age: int
    active: bool

# dataclass オブジェクト同士の比較はフィールドの値をみる
assert User("hoge", 20, active=True) == User("hoge", 20, active=True)

# 一般のクラスの場合
class C:
    def __init__(self, name: str, age: int, active: bool) -> None:
        self.name = name
        self.age = age
        self.active = active

assert C("hoge", 20, active=True) != C("hoge", 20, active=True)

### dataclass のマッチング ###

自動的に生成される `__match_args__` 属性により、match - case 文のクラスパターンで引数の名前を省略できる（つまり位置引数が使える）。以下のコードでは、各 case ブロックのクラスパターンの引数にキーワード引数を使用せず、位置引数としてキャプチャパターン `x`, `y` を使用している。

In [ ]:
from dataclasses import dataclass

@dataclass
class Point:
    x: float
    y: float

def f(point):
    match point:
        case Point(x, y) if x == y:
            print(f"Y=X at {x}")
        case Point(x, y):
            print("Not on the diagonal")

if __name__ == "__main__":
    f(Point(1.0, 1.0))
    f(Point(2.0, 1.0))

Y=X at 1.0
Not on the diagonal


### dataclass() の引数 ###

dataclass() デコレーターの引数は、すべてキーワード専用である。

| 引数 | 意味 | デフォルト値  |
|:--|:--|:-:|
| `init` | `True` の場合、`__init__()` メソッドが生成される | `True` |
| `repr` | `True` の場合、`__repr__()` メソッドが生成される | `True` |
| `eq` | `True` の場合、`__eq__()` メソッドが生成される | `True` |
| `match_args` | `True` の場合、`__match_args__` 属性が生成される（Python 3.10 で追加） | `True` |
| `order` | `True` の場合、`__lt__()`、`__le__()`、`__gt__()`、`__ge__()` メソッドが生成される | `False` |
| `frozen` | `True` の場合、フィールドへの代入は例外を生成する | `False` |
| `unsafe_hash` | `False` の場合、`eq` と `frozen` が両方とも `True` なら `__hash__()` メソッドを生成する | `False` |
| `kw_only` | `True` の場合、全てのフィールドはキーワード専用となる（Python 3.10 で追加） | `False` |
| `slots` | `True` の場合、`__slots__` 属性を持つ新しいクラスを返す（Python 3.10 で追加） | `False` |

`__init__()`、`__repr__()`、`__eq__()` メソッドを自分で定義したときは、`dataclass()` がその定義を上書きすることはない（引数 `init`、`repr`、`eq` は無視される）。

デフォルトでは、`__init__()` メソッドに渡された引数のリストからタプルを生成して、`__match_args__` 属性をそのタプルで定義する。もし `match_args` が `False` だったり、すでに `__match_args__` がクラスに定義されていた場合には `__match_args__` は生成されない。

もし `order` が `True` で、`eq` に `False` を指定すると、`ValueError` 例外が発生する。また `order` が `True` で、`__lt__()`、`__le__()`、`__gt__()`、`__ge__()` メソッドが既に定義されていると、`TypeError` 例外が発生する。

`@dataclass(frozen=True)` とすると、デフォルトでは `__hash__()` メソッドが生成され、dataclass のインスタンスはハッシュ可能となるから、set 型の要素や dict 型のキーにできる。

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class User:
    name: str
    active: bool = False

user = User("hoge")
try:
    user.active = True  # フィールドへの代入時にエラーが発生
except AttributeError as err:
    print(f"{type(err).__name__}: {err}")

users = {user}  # 集合の要素にできる
users.add(User("hoge"))  # フィールドの値が一致するオブジェクトは重複する要素とみなされる
assert len(users) == 1

# 辞書のキーに指定できる
user_id = {user: 1000}

FrozenInstanceError: cannot assign to field 'active'


### 初期化後の処理 ###

`dataclass()` デコレーターは、`__init__()` メソッドを生成するときに `__post_init__()` メソッドが既に定義されていると、`__init__()` の処理の最後に `__post_init__()` の呼び出しを追加する。これにより、`__init__()` によるフィールドの初期化を確実に済ませた後に、追加的な処理を `__post_init__()` メソッドに行わせることができる。dataclass の `__init__()` メソッドを自分で定義するよりは、`__post_init__()` メソッドを定義したほうが間違いがない。

たとえば、次のコードで `__post_init__()` はフィールドの値をチェックする:

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class User:
    name: str
    account_number: int

    def __post_init__(self):
        if self.account_number < 0:
            raise ValueError("cannot assign negative values to field 'account_number'.")
        if self.account_number >= 10000000:
            raise ValueError("cannot assign more than 8 digits to field 'account_number'.")

try:
    user = User("hoge", 12345678) # 8桁以上の口座番号を指定するとエラー
except ValueError as err:
    print(f"{type(err).__name__}: {err}")

ValueError: cannot assign more than 8 digits to field 'account_number'.


クラス変数宣言の型アノテーションに `dataclasses.InitVar` を指定した場合、そのクラス変数は一般的なフィールドとしては扱われず、**初期化限定変数**とされる。初期化限定変数は自動生成される `__init__()` メソッドと `__post_init__()` メソッドの引数に渡される。このため、初期化限定変数を宣言するときは、`__post_init__()` メソッドの定義に対応する引数を追加しないとエラーになる。

次の例では、`account_number_limt` をデフォルト値付きの初期化限定変数とし、インスタンス生成時に口座番号の桁数制限を変更できるようにしている。

In [ ]:
from dataclasses import dataclass, InitVar

@dataclass(frozen=True)
class User:
    name: str
    account_number: int
    account_number_limt: InitVar[int] = 10000000

    def __post_init__(self, account_number_limt):
        if self.account_number < 0:
            raise ValueError("cannot assign negative values to field 'account_number'.")
        if self.account_number >= account_number_limt:
            raise ValueError("cannot assign more than 8 digits to field 'account_number'.")

print(User("hoge", 12345678, account_number_limt=100000000))
assert User("foo", 1234, account_number_limt=10000) == User("foo", 1234, account_number_limt=9999)

User(name='hoge', account_number=12345678)


自動生成される `__repr__()` メソッドと `__eq__()` メソッドは初期化限定変数を無視する。このため、上記のコードにおいて初期化限定変数は `print()` の出力に現れないし、初期化限定変数の値の違いは `==` の比較で無視される。

### ユーティリティ関数 ###

`dataclasses` モジュールは、dataclass のインスタンス `obj` を操作する次のような便利な関数を提供している。

``` python
dataclasses.asdict(obj, *, dict_factory=dict)
```

`obj` を（ファクトリ関数 `dict_factory` を使い）辞書に変換する。フィールドの名前と値のペアから、キーと値のペアを作る。データクラス、辞書、リスト、タプルは再帰的に処理される。その他のオブジェクトは `copy.deepcopy()` でコピーされる。

``` python
dataclasses.astuple(obj, *, tuple_factory=tuple)
```

`obj` を（ファクトリ関数 `tuple_factory` を使い）タプルに変換する。フィールドの値から、タプルの要素を作る。データクラス、辞書、リスト、タプルは再帰的に処理される。その他のオブジェクトは `copy.deepcopy()` でコピーされる。

``` python
dataclasses.replace(obj, /, **changes)
```

`obj` と同じ型のオブジェクトを新しく作成し、フィールドを `changes` にある値で置き換える。`obj` のフィールドは変更されない。

In [ ]:
from dataclasses import dataclass, asdict, astuple, replace

@dataclass
class User:
    name: str
    id: int
    active: bool = False

user = User("hoge", 100)
assert asdict(user) == {"name": "hoge", "id": 100, "active": False}
assert astuple(user) == ("hoge", 100, False)
assert replace(user, active=True) == User(name='hoge', id=100, active=True)
assert user.active == False  # replece() は user のフィールド自体を変更しない

Python 3.13 からは、dataclass が `copy.replace()` 関数をサポートする。`dataclasses.replace()` 関数を使っても、`copy.replace()` 関数を使っても同じことができる。

### 値オブジェクト ###

dataclass の `frozen`、`slots` 引数と `dataclasses.replace()` 関数を組み合わせることで、次のような性質を持つオブジェクトを簡単に作成できる。

  1. オブジェクト同士の比較が可能で、同じ値を持つ場合は等しい。
  2. オブジェクトは不変である。
  3. 値の変更は、新しいオブジェクトを生成することによって処理される。

このような性質を全て持つオブジェクトを**値オブジェクト**（Value Object）と呼ぶ。実際、オブジェクト同士の比較は、dataclass で自動的に生成される `__eq__()` メソッドによって行われる。オブジェクトの不変性は、 dataclass の `frozen`、`slots` 引数を `True` に指定することで実現される。値の変更は、`dataclasses.replace()` 関数を使うようにすればよい。

値オブジェクトを使うと、コードの可読性が高くなり、また、バグが混入しにくくなるなどのメリットがあるとされる。

次のコードは、オンラインショップの商品に適用するクーポンを値オブジェクトとして設計している。クーポンオブジェクトのフィールドであるタイトル `title` 、割引率 `rate`、有効期限 `expiration` は変更不可であり（これは dataclass の引数 `frozen=True` の効果である）、フィールドの追加もできない（これは dataclass の引数 `slots=True` の効果である）。既存のクーポンの割引率や有効期限を変更するには、メソッドを呼び出して新たなクーポンオブジェクトを作成しなければならない。これによって、既存のクーポンの割引率や有効期限が意図せず変更されるなどのバグを回避できる。

In [ ]:
from dataclasses import dataclass, replace
from datetime import datetime


@dataclass(frozen=True, slots=True)
class Coupon:
    title: str
    rate: float
    expiration: datetime

    def update_rate(self, new: float):
        return replace(self, rate=new)

    def update_expiration(self, new: datetime):
        return replace(self, expiration=new)


c1 = Coupon("ブラックフライデー", 0.95, datetime(2023, 11, 27))
c2 = c1.update_expiration(datetime(2023, 12, 1))
assert c1 != c2 == Coupon("ブラックフライデー", 0.95, datetime(2023, 12, 1))
try:
    c1.rate = 0.095
except:
    print("Value Object は変更不可能です")

Value Object は変更不可能です


### dataclass を継承する列挙型 ###

列挙型は dataclass をサポートする。列挙型の定義で dataclass  1 つと `enum.Enum` を多重継承する場合、メンバーはその dataclass のインスタンスとなる。ただし、メンバーの定義では dataclass の位置引数をタプルで指定する。キーワード引数を渡す方法はないので、キーワード専用フィールドを持つ dataclass には対応していない。

In [ ]:
from dataclasses import dataclass, field
from enum import Enum

@dataclass
class CreatureDataMixin:
    size: str
    legs: int
    tail: bool = field(repr=False, default=True)

class Creature(CreatureDataMixin, Enum):
    DOG = "medium", 4
    GORILLA = "big", 4, False

assert isinstance(Creature.DOG, Creature) and isinstance(Creature.DOG, CreatureDataMixin)
assert Creature.DOG.size == "medium"  # フィールドは value 属性を経由せずにアクセスできる
assert Creature.DOG.tail  # デフォルト値
assert not Creature.GORILLA.tail
Creature.GORILLA  # tail フィールドは表示されない

Creature(size='big', legs=4)

### dataclass_transform ###

次のコードの `CustomerModel` クラスは属性に対する型アノテーションを付与し dataclass に似ているが、静的型チェックツールは `CustomerModel()` の引数の型をチェックしてくれない。

``` python
class ModelBase:
    def __init__(self, **kwargs) -> None:
        pass

class CustomerModel(ModelBase):
    id: int
    name: str

c = CustomerModel(id="1", name="hoge")  # 引数 id に文字列を指定しているが、型チェックはエラーにならない
```

原因は、`CustomerModel` が `__init__()` メソッドを定義せず、基底クラス `ModelBase` から継承する `__init__()` メソッドには引数に対する型アノテーションがないからである。

Python 3.11 からは、`typing.dataclass_transform()` デコレーターが使える。このデコレーターを基底クラスに付けると、対応する静的型チェックツールは派生クラスの属性に対する型アノテーションを利用するようになる。

``` python
from typing import dataclass_transform

@dataclass_transform()
class ModelBase:
    def __init__(self, **kwargs) -> None:
        pass

class CustomerModel(ModelBase):
    id: int
    name: str

c = CustomerModel(id="1", name="hoge")  # 型チェッカーは、引数 id に文字列を指定していることに対してエラーを発生する
```

`typing.dataclass_transform()` デコレーターは、実際に dataclass を生成しているわけではなく、クラスに `__dataclass_transform__` 属性を追加するだけである。型チェッカーは、`__dataclass_transform__` 属性からの情報に基づき dataclass のように型アノテーションの情報を元にした型チェックを行う。

上記のコードでは、基底クラスに `typing.dataclass_transform()` を付けたが、クラスに付けるデコレーターを適当に作成し、それに `typing.dataclass_transform()` を付けることでもよい。

``` python
from typing import dataclass_transform

class ModelBase:
    def __init__(self, **kwargs) -> None:
        pass

@dataclass_transform()
def create_model[T](cls: type[T]) -> type[T]:
    """CustomerModel に適用するだけのデコレーター"""
    ...
    return cls

@create_model
class CustomerModel(ModelBase):
    id: int
    name: str

c = CustomerModel(id="1", name="hoge")  # 型チェッカーは、引数 id に文字列を指定していることに対してエラーを発生する
```

`typing.dataclass_transform()` デコレーターは、Django 内蔵の O/Rマッパーのように、サードパーティー製ライブラリが dataclass と似た構造を持つクラスを提供していて、型チェックを行いたい場合に使用されることが想定されている。